In [5]:
import json
import os
import shutil
from tqdm import tqdm
from PIL import Image # Resim boyutlarını doğru almak için gerekli

# --- LÜTFEN SADECE BU 2 YOLU KENDİ BİLGİSAYARINIZA GÖRE DÜZENLEYİN ---

# 1. Balon veri setinin ana klasörü (Bu klasörün içinde /train/ ve /val/ olmalıdır)
# ÖRNEK YOL: r"C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\data\balloon"
BASE_DIR = r"C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\data\balloon" 

# 2. Yeni YOLO formatında çıktıların (images ve labels) geleceği temiz klasör
OUTPUT_YOLO_DIR = r"C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\balloon_YOLO_OUTPUT"

# --- SABİT TANIMLAMALAR ---
CLASS_NAMES = ['balloon']
CATEGORY_ID = 0 # Balon tek sınıf olduğu için YOLO ID'si 0'dır.

def convert_via_to_yolo(split_name):
    """via_region_data.json dosyasından etiketleri okur ve YOLO formatına dönüştürür."""
    print(f"\n--- {split_name.upper()} Veri Seti Dönüştürülüyor ---")
    
    # Giriş yolları
    split_path = os.path.join(BASE_DIR, split_name)
    json_file = os.path.join(split_path, 'via_region_data.json')
    
    # Çıktı yolları
    output_img_dir = os.path.join(OUTPUT_YOLO_DIR, 'images', split_name)
    output_label_dir = os.path.join(OUTPUT_YOLO_DIR, 'labels', split_name)
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    if not os.path.exists(json_file):
        print(f"HATA: {json_file} bulunamadı.")
        return

    with open(json_file, 'r') as f:
        data = json.load(f)

    # JSON yapısı: Resimler listesi yerine resim adları doğrudan anahtar (key) olarak kullanılıyor.
    # Bu yüzden 'data.items()' ile doğrudan okuyoruz.
    
    # 1. Resim verilerini işle
    for filename_with_size, v_data in tqdm(data.items(), desc=f"İşleniyor {split_name}"):
        
        filename = v_data['filename']
        img_path = os.path.join(split_path, filename)

        if not os.path.exists(img_path):
             # Dosya yolunda problem varsa atla
             continue

        # Resim boyutunu PIL ile alalım (Normalize etmek için gerekli)
        try:
            img = Image.open(img_path)
            img_w, img_h = img.size
        except Exception as e:
            print(f"UYARI: {filename} boyutu alınamadı. Atlanıyor.")
            continue
            
        
        # Etiketlerin kaydedileceği TXT dosyası
        label_file_path = os.path.join(output_label_dir, filename.replace('.jpg', '.txt'))
        
        # 2. Etiketleri oluştur (Her seferinde dosyayı sıfırlar)
        with open(label_file_path, 'w') as out_f:
            
            if 'regions' in v_data and v_data['regions']:
                
                # Bizden Detection (BBOX) istendiği için, poligonları çevreleyen kutuyu bulacağız.
                for region_id, region in v_data['regions'].items():
                    
                    if region['shape_attributes']['name'] == 'polygon':
                        
                        all_x = region['shape_attributes']['all_points_x']
                        all_y = region['shape_attributes']['all_points_y']
                        
                        # Minimum Çevreleyen Kutu (Bounding Box) hesaplama
                        x_min = min(all_x)
                        y_min = min(all_y)
                        x_max = max(all_x)
                        y_max = max(all_y)
                        
                        w = x_max - x_min
                        h = y_max - y_min
                        
                        # YOLO formatına çevirme (Normalize edilmiş merkez koordinatları ve boyut)
                        x_center = (x_min + w / 2) / img_w
                        y_center = (y_min + h / 2) / img_h
                        w_norm = w / img_w
                        h_norm = h / img_h
                        
                        # Sınıf ID'si (Balon için 0)
                        yolo_line = f"{CATEGORY_ID} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n"
                        out_f.write(yolo_line)

        # 3. Resmi yeni YOLO klasörüne kopyala
        target_img_path = os.path.join(output_img_dir, filename)
        shutil.copy2(img_path, target_img_path)
        
    print(f"{split_name.upper()} dönüşümü bitti.")


# --- ÇALIŞTIRMA KISMI (YOLOLARINIZI DÜZENLEDİKTEN SONRA BU KISMI ÇALIŞTIRIN) ---

if __name__ == "__main__":
    
    # 🚨 Gerekliyse PIL (Pillow) kurun!
    # print("Pillow kontrol ediliyor...")
    # try:
    #     from PIL import Image
    # except ImportError:
    #     print("Pillow kurulu değil. Kurulum: pip install Pillow")
    #     exit()
        
    # Önceki çıktıları temizle
    if os.path.exists(OUTPUT_YOLO_DIR):
        shutil.rmtree(OUTPUT_YOLO_DIR)
        
    convert_via_to_yolo('train')
    convert_via_to_yolo('val')
    
    print("\n--- TÜM DÖNÜŞÜM BAŞARILI! ---")
    print(f"YOLO formatı hazırlandı: {OUTPUT_YOLO_DIR}")


--- TRAIN Veri Seti Dönüştürülüyor ---


İşleniyor train: 100%|████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 61.31it/s]


TRAIN dönüşümü bitti.

--- VAL Veri Seti Dönüştürülüyor ---


İşleniyor val: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 53.71it/s]

VAL dönüşümü bitti.

--- TÜM DÖNÜŞÜM BAŞARILI! ---
YOLO formatı hazırlandı: C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\balloon_YOLO_OUTPUT
